# hmm applied to scanpaths

In [47]:
# imports
import hmmlearn
from hmmlearn import hmm
from hmmlearn.vhmm import VariationalCategoricalHMM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm
from matplotlib.animation import FuncAnimation
import itertools
import random
import seaborn as sns
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


from src.d00_utils.TaskObjects import *
from src.d03_processing.fixations.FixationProcessor import FixationProcessor
from src.d03_processing.TimepointProcessor import TimepointProcessor
from src.d01_data.fetch.fetch_viewings import fetch_viewings
from src.d01_data.fetch.fetch_trials import fetch_trials
from src.d01_data.fetch.fetch_fixations import fetch_fixations
from src.d03_processing.aoi import collision_sphere_radius
from src.d03_processing.feature_extract.to_viewing import to_viewing
from src.d03_processing.fixations.FixAlgos import *
from src.d03_processing.feature_calculate.viewing_compare_calcs import ea_td
from src.d03_processing.feature_calculate.transition_calculations import *
from src.d04_wrangling.add_ppt_info import get_ppts_by_group
from sklearn.preprocessing import LabelEncoder
from src.d03_processing.feature_calculate.fix_sacc_calculations import n_fix, dwell_time

In [48]:
def get_object_position(obj, trial, prepostshift):
    for i in [1, 2, 3, 4]:
        # If any matches were found, return the corresponding x and z values
        if obj == trial[f'obj{i}_name'].values[0]:
            return np.array([trial[f'obj{i}_{prepostshift}_x'].values[0], trial[f'obj{i}_{prepostshift}_z'].values[0]])

def convert_AOIs(fix_df, trial=None, prepostshift=None, encoder=False):

    # get trial row if none
    if trial is None:
        trial_id = fix_df.trial_id.values[0]
        trial = fetch_trials("all", trial_ids=[trial_id], suppress_print=True, remove_training_trials=False, practice=[False, True])

    # get viewing type
    if prepostshift is None:
        prepostshift = 'preshift' if 'enc' in fix_df.viewing_id.values[0] else 'postshift'


    new_objects = fix_df.object.copy()
    pd.options.mode.chained_assignment = None  # default='warn'

    # change names of objects to set AOI names
    new_objects[new_objects.isin(TaskObjects.off_table)] = 'External'
    new_objects[new_objects.str.contains('Table')] = 'Table'
    new_objects[new_objects == TaskObjects.invisible_object] = 'Previous'
    new_objects[new_objects == trial.obj1_name.values[0]] = 'Moved'

    # for objs 2 to 4, change name according to distance from moved object
    obj1_loc = np.array([trial[f'obj1_{prepostshift}_x'].values[0], trial[f'obj1_{prepostshift}_z'].values[0]])
    objs2_to_4 = np.unique(new_objects[(new_objects != 'Moved') & new_objects.isin(TaskObjects.array_objects)])
    objs2_to_4_distance_dict = {'object': [], 'distance': []}
    for obj in objs2_to_4:
        objs2_to_4_distance_dict['object'].append(obj)
        obj_loc = get_object_position(obj, trial, prepostshift)
        objs2_to_4_distance_dict['distance'].append(np.linalg.norm(obj_loc - obj1_loc))
    objs_df = pd.DataFrame(objs2_to_4_distance_dict)
    objs_df = objs_df.sort_values(by='distance').reset_index(drop=True)
    for obj in objs2_to_4:
        new_objects[new_objects == obj] = f"Obj{objs_df.index.values[objs_df.object == obj][0]+2}"

    # optionally convert to integer labels
    if encoder:
        new_objects = aoi_label_encoder(new_objects)
    fix_df.object = new_objects
    return fix_df

def aoi_label_encoder(fixation_sequence):
    # Define your custom mapping
    label_mapping = {"Previous": 0, "Moved": 1, "Obj2": 2, "Obj3": 3, "Obj4": 4, "Table": 5, "External": 6}

    # Encode your labels using the mapping
    encoded_labels = [label_mapping[label] for label in fixation_sequence]

    return encoded_labels

def split_fixations(fix_df, bin_size=50):
    fixations = fix_df.copy().sort_values(by='start_time')
    fixation_sequence = []

    for index, row in fixations.iterrows():
        aoi, duration = row['object'], row['duration_time']

        while duration >= bin_size:
            fixation_sequence.append(aoi)
            duration -= bin_size

    return np.array(fixation_sequence)

In [49]:
# get data
pids_dict = get_ppts_by_group()
young_pids = pids_dict['Younger']
old_pids = pids_dict['Older']
mci_pids = pids_dict['MCI+']
# cutoff = 7000   # ms
pid_groups = [young_pids, old_pids, mci_pids]
group_names = ['Younger', 'Older', 'MCI']
time_bin_size = 1000 # ms
# n_bins = int((cutoff / time_bin_size )) + 1
viewing_type = 'ret'

In [52]:
model.

AttributeError: 'MultinomialHMM' object has no attribute 'print'

In [53]:
plt.close('all')

grouped_all_ps = []
for g in range(len(pid_groups)):
    g = 1
    print(group_names[g], '*****************************************************************')
    pids = pid_groups[g]
    n_pids = len(pids)

    n_pids = 3
    for p in range(n_pids):
        # p = 3
        print(pids[p])
        all_fixsac = fetch_fixations(pids[p], practice=True, suppress_print=True)
        all_fixsac = all_fixsac[all_fixsac.viewing_id.str.contains(viewing_type)]
        all_fix = all_fixsac[all_fixsac.fixation_or_saccade == 'fixation'].reset_index(drop=True).sort_values(by='start_time')
        all_trials = fetch_trials(pids[p], practice=[True, False], suppress_print=True)
        # print(all_trials.shape)
        trials_fix = np.unique(all_fix.trial_id)
        n_trials = len(trials_fix)
        n_valid_trials = n_trials

        n_trials = 10 if n_trials > 10 else n_trials
        for t in range(n_trials):
            # t = 15
            trial_id = trials_fix[t]
            trial = all_trials[all_trials.trial_id == trial_id].reset_index(drop=True)
            fixations = all_fix[all_fix.trial_id == trial_id].reset_index(drop=True).sort_values(by='start_time')
            # print("fixations", fixations.shape)
            # print("trial", trial_id)
            if len(trial) < 1:
                n_valid_trials -= 1
                continue
            if len(fixations) < 1:
                n_valid_trials -= 1
                continue

            # external fixations only
            ext_fix = external_fixations(fixations)

            # convert to areas of interest
            ext_fix = convert_AOIs(ext_fix, trial, 'postshift', encoder=True)

            # split into sequence
            aoi_sequence = split_fixations(ext_fix)
            print(aoi_sequence)

            # the number of states in our HMM
            n_states = 2

            # initialize the HMM
            model = hmm.MultinomialHMM(n_components=n_states, algorithm='viterbi', n_iter=100)

            # fit the HMM to our data
            model.fit(aoi_sequence.reshape(-1, 1))

            print(model.decode(aoi_sequence.reshape(-1, 1)))
                        # VariationalCategoricalHMM(n_components=2)
# print("test post")

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Older *****************************************************************
alloeye_1
- dataframe returned in 0.10175609588623047
[6 6 6 6 6 5 5 5 5 2 1 1 1 1 1 2 2 2 2 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3
 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 5 5 5 5
 5 5 5 5 1 1 1 1 1]


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows

(-11.925199103046365, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64))
[3 5 5 1 1 3 3 3 3 4 4 5 5 1 1 1 1 1 1 1 1 1 3 3 3 4 4 4 4 5 5 5 5 5 5 5 5
 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 1 1 1 1 1 1 1 5 5 5 5
 5 3 3 3 3 1 1 1 1 1 2 2 2 1 1 1 3 3 3 3 3 3 3 3 3 3 3]
(-5.290289513187561, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

ValueError: startprob_ must sum to 1 (got nan)